In [ ]:
import time, sys
from typing import Type, List, Dict, Tuple, Set
import argparse
try:
    from sklearn.externals import joblib
    from sklearn.externals.joblib import parallel_backend, Parallel, delayed
except ImportError:
    import joblib
    from joblib import parallel_backend, Parallel, delayed
    
import pandas as pd
import json, ijson
import os, sys, uuid
from pykalman import KalmanFilter
from PIL import Image
import math
import ast
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from shapely.geometry import Polygon, Point
from shapely.geometry import Polygon
from matplotlib.backends.backend_pdf import PdfPages
from geopy.distance import geodesic, distance
from geopy import Point
from shapely.geometry import Point, Polygon as ShapelyPolygon
from matplotlib.patches import Polygon as MplPolygon

from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from datetime import datetime
import random

from matplotlib.patches import Polygon
import ast

from collections import defaultdict
import pyarrow.parquet as pq

import pickle
from ast import literal_eval

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as mcolors

import matplotlib.image as mpimg

from tqdm.notebook import tqdm, trange
from scipy.optimize import minimize
from scipy.optimize import least_squares

from os import walk
from os import listdir
from os.path import isfile, join, isdir

import scipy.optimize as opt
from shapely.geometry import Polygon
from shapely.geometry import Polygon

from PIL import Image
from matplotlib.patches import Polygon

from sklearn import model_selection
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

import warnings
warnings.filterwarnings("ignore")

In [ ]:
start =time.time()
CHANNELS = [37,38,39]
N_ESTIMATORS = 100
MISSING_VALUE = -100
DEBUG_LOGGING = False
S3_CACHING_BUCKET = 'cognosos-ml-data'

In [ ]:
def parse_scan_data_woc(scan: List[Dict]) -> Dict:
    # Parse each scan to get maximum reading for each MAC address in specified channels
    readings_by_mac_addr_and_channel = defaultdict(list)
    for beacon_reading in scan:
        if beacon_reading['channel'] in CHANNELS:
            mac_addr = beacon_reading['macHex']
            readings = beacon_reading['readings']
            readings_by_mac_addr_and_channel[mac_addr] += readings
    return {mac_addr: int(max(readings)) for mac_addr, readings in readings_by_mac_addr_and_channel.items() if readings}


def process_training(data_filepath: str) -> List[Dict]:
    X = []

    # parse it incrementally
    with open(data_filepath, 'r') as f:
        # reads the JSON incrementally
        objects = ijson.items(f, 'item') 

        print('Done loading JSON incrementally')

        for scan in objects:
            
            Zone_id = str(scan['zoneId'])
            Room_name = str(scan['zoneName'])
            parent_zone_id = str(scan['parentZoneId'])
            tagId = scan['tagId']
            timestamp = scan['rxAt']
            scan_readings: List[Dict] = scan['scandata']
            
            row = parse_scan_data_woc(scan_readings) 

            row.update({
                'Zone_id': Zone_id,
                'Room_name': Room_name,
                'parent_zone_id': parent_zone_id,
                'tagId': tagId,
                'timestamp': timestamp,
            })

            if row:
                X.append(row)

    print('Done processing data')

    return X

In [ ]:
def extract_values(scan_data):

    if scan_data is None:
        return []
    return [
        {'macHex': entry['macHex'], 'channel': entry['channel'], 'readings': [entry['rssi'][0]]}
        for entry in scan_data if 'macHex' in entry and 'rssi' in entry
    ]

def parse_scan_data(scan: List[Dict]) -> Dict:

    readings_by_mac_addr_and_channel = defaultdict(list)
    for beacon_reading in scan:
        if beacon_reading['channel'] in CHANNELS:
            mac_addr = beacon_reading['macHex']
            readings = beacon_reading['readings']
            channel = beacon_reading['channel']
            readings_by_mac_addr_and_channel[f'{mac_addr}'] += readings#-{channel}
    return {mac_addr: max(readings) for mac_addr, readings in readings_by_mac_addr_and_channel.items() if len(readings) > 0 }

In [ ]:
def train_variable(X_train, y_train_floor, y_train, save_models=False):
    
    floor_pipeline = Pipeline([
        ('rf', RandomForestClassifier(random_state=42))
    ])

    floor_pipeline.fit(X_train, y_train_floor)

    clf_floor = floor_pipeline.named_steps['rf']

    clf_rooms = {}

    selected_features = {}

    for floor_num, samples in X_train.groupby(y_train_floor):
        
        floor_labels = y_train[samples.index]

        non_all_neg_120_columns = samples.columns[~np.all(samples == -120, axis=0)]

        selected_samples = samples[non_all_neg_120_columns]

        classifier = RandomForestClassifier(n_estimators=200, random_state=100)

        classifier.fit(selected_samples, floor_labels)

        clf_rooms[str(floor_num)] = classifier

        selected_features[str(floor_num)] = selected_samples.columns.tolist()

    if save_models:
        model = {
        'selected_features': selected_features,
        'clf_rooms': clf_rooms,
        'clf_floor': clf_floor
        }
        joblib.dump(model, 'Hier_Features.joblib')
        
    return selected_features, clf_rooms, clf_floor

def predict_variable(X_test, clf_floor, clf_rooms, selected_features):
    
    predicted_floors = clf_floor.predict(X_test)

    predictions = []
    for floor_num, sample in zip(predicted_floors, X_test.values):
        classifier = clf_rooms[str(floor_num)]

        selected_names = selected_features[floor_num]

        selected_sample = sample[X_test.columns.isin(selected_names)].reshape(1, -1)

        predicted_room = classifier.predict(selected_sample)[0]
#         predicted_room = predicted_room.astype(str)
        predictions.append(predicted_room)

    return predictions, predicted_floors

In [ ]:
def create_digital_twin(Anchor_point_location_file, ground_truth_file_location, map_file_location):

    anchor_df = pd.read_csv(Anchor_point_location_file)
    anchor_df["x"] = anchor_df["x"].astype(int)
    anchor_df["y"] = anchor_df["y"].astype(int)
    
    # I ADD THIS TO Ensure MAC addresses are strings and zero-padded to length 12
    anchor_df['Mac'] = anchor_df['Mac'].astype(str).str.zfill(12)
    
    macLists = anchor_df['Mac'].to_list()

    ground_truth_df = pd.read_csv(ground_truth_file_location)
    ground_truth_df["Zone_id"] = ground_truth_df["Zone_id"].astype(str)
    
    #create a empty map with 0s for future calculation
    map_ = np.zeros((65,28))

    plt.figure(figsize=(12, 6))
    
    image = Image.open(map_file_location)
    
    plt.scatter(anchor_df.x,anchor_df.y, color='blue', s=50, edgecolors='black', label='Beacons', marker='o', alpha=0.6)

#     plt.scatter(ground_truth_df["x"], ground_truth_df["y"], color='red', s=20, label='Ground Truth', marker='^')
#     for i, label in enumerate(ground_truth_df['Room_name']):  
#         plt.text(ground_truth_df['x'][i], ground_truth_df['y'][i], label, fontsize=9, color='w', ha='right', va='bottom')
    plt.imshow(image, extent=[0, 65, 0, 28], aspect='auto')

    plt.xlim(0, 65)
    plt.ylim(0, 28)

    plt.grid(True)
    plt.xticks([i for i in range(0, 65, 5)])
    plt.yticks([i for i in range(0, 28, 4)])
    plt.xlabel('x', fontsize=14)
    plt.ylabel('y', fontsize=14)
    plt.title("Beacon distribution in meters")
    plt.legend()
    plt.savefig('beacon_map_cognosos.png')

    return anchor_df, ground_truth_df, map_

In [ ]:
beacon_file = 'ground_truth/Beacon_map_cognosos_flr3.csv'
ground_truth_file = "ground_truth/Ground_truth_Mar25.csv"
map_file = 'ground_truth/Cognosos_view.png'

anchor_point_df, ground_truth_df, map_ = create_digital_twin(beacon_file, ground_truth_file, map_file)

In [ ]:
# anchor_point_df

In [ ]:
def filter_valid_features(row, df1):

    valid_features = {}
    
    for mac in df1['Mac']:
       
        if mac in row.index and isinstance(row[mac], (int, float)) and row[mac] != -100:
            valid_features[mac] = row[mac]
    
    return valid_features

def convert_coordinates(coord_str):
    if isinstance(coord_str, str):
       
        try:
            coord_str = coord_str.strip("[]")
            elements = coord_str.split()
            return [float(elem) for elem in elements] 
        except ValueError:
            pass  

        try:
            coord_str = coord_str.replace(" ", ",")
            coord_str = coord_str.replace(",,", ",")
            coord_str = coord_str.strip(',')
            return ast.literal_eval(coord_str)
        except (ValueError, SyntaxError) as e:
            print(f"Error processing coordinate string: {coord_str}")
            return None
    else:
        return coord_str

In [ ]:
def plot_predicted_all(result, ground_truth_df, map_file_location, output_file="compare_plot_MLE_NLOS.png"):
    results = []
    total_points_mle = 0
    total_points_Optimisation = 0
    total_points_fuse = 0

    total_inside_mle = 0
    total_inside_Optimisation = 0
    total_inside_fuse = 0

    merged_df = pd.merge(result, ground_truth_df, on=["Zone_id", "Room_name"], how="left")
    unique_rooms = merged_df['Room_name'].unique()

    n_rows = math.ceil(len(unique_rooms) / 2)
    fig, axes = plt.subplots(n_rows, 2, figsize=(14, 3 * n_rows))
    axes = axes.flatten()

    for i, room_name in enumerate(unique_rooms):
        room_data = merged_df[merged_df['Room_name'] == room_name]
        has_fused = 'Predicted_NLOS' in room_data.columns

        zone = room_data["Zone_id"].iloc[0]
        room_type = room_data["Room_Type"].iloc[0]
        room_box = room_data.iloc[0]

        x_coords = [room_box.get(f'x{i+1}', None) for i in range(8) if pd.notnull(room_box.get(f'x{i+1}', None))]
        y_coords = [room_box.get(f'y{i+1}', None) for i in range(8) if pd.notnull(room_box.get(f'y{i+1}', None))]

        coordinates = list(zip(x_coords, y_coords))
        polygon = Polygon(coordinates)

        if not polygon.is_valid:
            print(f"Invalid polygon for '{room_name}', attempting to fix with buffer(0).")
            polygon = polygon.buffer(0)

        # Parse MLE predictions
        x_pred_mle, y_pred_mle = [], []
        for coord in room_data["Predicted_MLE"]:
            try:
                coord = ast.literal_eval(coord) if isinstance(coord, str) else coord
                x_pred_mle.append(float(coord[0]))
                y_pred_mle.append(float(coord[1]))
            except:
                print(f"Invalid MLE coord in '{room_name}': {coord}")

        # Parse Optimisation predictions
        x_pred_Optimisation, y_pred_Optimisation = [], []
        for coord in room_data["Predicted_Optimisation"]:
            try:
                coord = ast.literal_eval(coord) if isinstance(coord, str) else coord
                x_pred_Optimisation.append(float(coord[0]))
                y_pred_Optimisation.append(float(coord[1]))
            except:
                print(f"Invalid Optimisation coord in '{room_name}': {coord}")

        # Parse Fused predictions only if available
        x_pred_fuse, y_pred_fuse = [], []
        if has_fused:
            for coord in room_data["Predicted_NLOS"]:
                try:
                    coord = ast.literal_eval(coord) if isinstance(coord, str) else coord
                    x_pred_fuse.append(float(coord[0]))
                    y_pred_fuse.append(float(coord[1]))
                except:
                    print(f"Invalid NLOS coord in '{room_name}': {coord}")

        inside_count_mle = sum(1 for x, y in zip(x_pred_mle, y_pred_mle) if Point(x, y).within(polygon))
        inside_count_Optimisation = sum(1 for x, y in zip(x_pred_Optimisation, y_pred_Optimisation) if Point(x, y).within(polygon))
        inside_count_fuse = sum(1 for x, y in zip(x_pred_fuse, y_pred_fuse) if Point(x, y).within(polygon)) if has_fused else 0

        total_points_mle += len(x_pred_mle)
        total_inside_mle += inside_count_mle

        total_points_Optimisation += len(x_pred_Optimisation)
        total_inside_Optimisation += inside_count_Optimisation

        if has_fused:
            total_points_fuse += len(x_pred_fuse)
            total_inside_fuse += inside_count_fuse

        percentage_inside_mle = (inside_count_mle / len(x_pred_mle)) * 100 if x_pred_mle else 0
        percentage_inside_Optimisation = (inside_count_Optimisation / len(x_pred_Optimisation)) * 100 if x_pred_Optimisation else 0
        percentage_inside_fuse = (inside_count_fuse / len(x_pred_fuse)) * 100 if has_fused and x_pred_fuse else 0

        results.append({
            "Zone_id": zone,
            'Room_name': room_name,
            "Room_Type": room_type,
            'MLE_Accuracy': percentage_inside_mle,
            'Optimisation_Accuracy': percentage_inside_Optimisation,
            'NLOS_Accuracy': percentage_inside_fuse if has_fused else None,
            'MLE_Inside_Points': inside_count_mle,
            'Optimisation_Inside_Points': inside_count_Optimisation,
            'NLOS_Inside_Points': inside_count_fuse if has_fused else None,
            'Total_Points': len(x_pred_mle),
        })

        # Plot
        ax = axes[i]
        image = mpimg.imread(map_file_location)
        ax.imshow(image, extent=[0, 65, 0, 28], aspect='auto')
        ax.plot(x_coords + [x_coords[0]], y_coords + [y_coords[0]], 'r-', label='Room Boundary')
        ax.scatter(x_pred_mle, y_pred_mle, color='blue', s=8, label='MLE')
        ax.scatter(x_pred_Optimisation, y_pred_Optimisation, color='green', s=8, label='Optimisation')
        if has_fused:
            ax.scatter(x_pred_fuse, y_pred_fuse, color='red', s=8, label='NLOS')

        ax.set_xlim([0, 65])
        ax.set_ylim([0, 28])
        ax.set_xlabel("X Coordinate")
        ax.set_ylabel("Y Coordinate")
        title_str = f"{room_name} - MLE: {percentage_inside_mle:.1f}%, Optimisation: {percentage_inside_Optimisation:.1f}%"
        if has_fused:
            title_str += f", NLOS: {percentage_inside_fuse:.1f}%"
        ax.set_title(title_str)
        ax.legend(loc='lower left', bbox_to_anchor=(0, 0), ncol=2)

    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    overall_mle_accuracy = (total_inside_mle / total_points_mle) * 100 if total_points_mle > 0 else 0
    overall_Optimisation_accuracy = (total_inside_Optimisation / total_points_Optimisation) * 100 if total_points_Optimisation > 0 else 0
    overall_fuse_accuracy = (total_inside_fuse / total_points_fuse) * 100 if total_points_fuse > 0 else 0

    print(f"\nOverall MLE Accuracy: {overall_mle_accuracy:.2f}%")
    print(f"Overall Optimisation Accuracy: {overall_Optimisation_accuracy:.2f}%")
    if total_points_fuse > 0:
        print(f"Overall NLOS Accuracy: {overall_fuse_accuracy:.2f}%")

    accuracy_df = pd.DataFrame(results)
    plt.tight_layout()
    plt.savefig(output_file, format="png")
    plt.show()

    return accuracy_df

In [ ]:
def read_MLE_data_survey_portal(df, ground_truth_df, anchor_point_df, export_unheard=False, export_path="unheard_anchor_points.csv"):

    # I ADD THIS TO Ensure MAC addresses are strings and zero-padded to length 12
    anchor_point_df['Mac'] = anchor_point_df['Mac'].astype(str).str.zfill(12)

    data_set_df = pd.DataFrame()
    merged_df = pd.merge(df, ground_truth_df, on=["Zone_id", 'Room_name'], how='inner').drop(['parent_zone_id'], axis=1)
    zones = df['Zone_id']
    heard_anchor_points = []

    for mac_addr in anchor_point_df['Mac']:
        if mac_addr in merged_df.columns:
            data_set_df[mac_addr] = merged_df[mac_addr]
            heard_anchor_points.append(mac_addr)

    heard_anchor_point_df = anchor_point_df[anchor_point_df['Mac'].isin(heard_anchor_points)].reset_index(drop=True)
    unheard_anchor_point_df = anchor_point_df[~anchor_point_df['Mac'].isin(heard_anchor_points)].reset_index(drop=True)

    heard_anchor_points_coord = heard_anchor_point_df[['x', 'y']].values

    data_set_df["Zone_id"] = merged_df["Zone_id"]
    data_set_df["Room_name"] = merged_df["Room_name"]
    data_set_df["tagId"] = merged_df["tagId"]
    data_set_df["timestamp"] = merged_df["timestamp"]
    if "channel" in merged_df.columns:
        data_set_df["channel"] = merged_df["channel"]
    
    data_set_df["x"] = merged_df["x"]
    data_set_df["y"] = merged_df["y"]

    return data_set_df, heard_anchor_points_coord, unheard_anchor_point_df

In [ ]:
filepath = "data/Asset/Open_area_with_ground_truth/data_asset_open_area_with_ground_truth_Nov24.json"
filename = os.path.basename(filepath)

X1 = process_training(filepath)
df1 = pd.DataFrame(X1)
df1 = df1.fillna(MISSING_VALUE)  

float_cols = df1.select_dtypes(include=['float']).columns
df1[float_cols] = df1[float_cols].astype(np.int8)
df1['timestamp'] = pd.to_datetime(df1['timestamp'], utc=True, errors='coerce')
df1 = df1.sort_values(by='timestamp')

ordered_columns = ['timestamp', 'tagId', 'Zone_id', 'Room_name', "parent_zone_id"]

columns = [col for col in anchor_point_df.Mac.unique().tolist() if col not in ordered_columns]
new_column_order = columns + ordered_columns
df1 = df1.reindex(columns=new_column_order)

df1 = df1.reset_index(drop=True)
df1['Room_name'] = df1['Room_name'].str.split('-').str[-1].str.strip()

# Fix specific zone_id
df1.loc[df1['Zone_id'] == "30598", 'Zone_id'] = "30539"

# Beacon processing
beacon_cols = [col for col in df1.columns if str(col).startswith('0')]
df1 = df1.fillna(MISSING_VALUE)
df1['beacon_count'] = (df1[beacon_cols] != -100).sum(axis=1)

print(df1.shape)

df1 = df1[df1['beacon_count'] >= 5]
print(df1.shape)

### Remove some timestamp as I forgot to turn off the survey when I move the tag

In [ ]:
df1['timestamp'] = pd.to_datetime(df1['timestamp'])

# Define the time range to remove
start_time = pd.to_datetime("18:11:00").time()
end_time = pd.to_datetime("18:24:00").time()

# Filter out rows where the timestamp's time is in that range
df = df1[~df1['timestamp'].dt.time.between(start_time, end_time)]
df.shape, df1.shape

### I need to correct the label as I was in Lab area 1 but took the survey data for Laba rea 2

In [ ]:
wrong_zone = "30536"
correct_zone = "30537"

wrong_room = "Lab Area 1"
correct_room = "Lab Area 2"


df['Zone_id'] = df['Zone_id'].replace({
    wrong_zone: 'TEMP_ZONE',
    correct_zone: wrong_zone
}).replace({'TEMP_ZONE': correct_zone})

df['Room_name'] = df['Room_name'].replace({
    wrong_room: 'TEMP_ROOM',
    correct_room: wrong_room
}).replace({'TEMP_ROOM': correct_room})

In [ ]:
df.loc[(df['tagId'] == '2037320') & (df['Room_name'] == 'Lab Area 2'), ['tagId', 'Room_name']] = \
    ['2037321', 'Lab Area 1']
df.loc[(df['tagId'] == '2040157') & (df['Room_name'] == 'Lab Area 2'), ['tagId', 'Room_name']] = \
    ['2040158', 'Lab Area 1']
df.loc[(df['tagId'] == '2038775') & (df['Room_name'] == 'Lab Area 2'), ['tagId', 'Room_name']] = \
    ['2038776', 'Lab Area 1']

df.loc[df['Room_name'] == 'Lab Area 1', 'Zone_id'] = "30536"

In [ ]:
df.to_csv("data/Asset/data_asset_open_area_with_ground_truth_Nov24.csv", index=False)

In [ ]:
df1= df.copy()

In [ ]:
df1['beacon_count'].max(), df1['beacon_count'].min()

In [ ]:
df1.Room_name.nunique()

In [ ]:
df1.shape

## REMOVE ALL ROWS < -90

### Read the data for walk around with a plastic box of tags

In [ ]:
# df1= pd.read_csv("data/Asset/data_asset_tag_stay_still_combine_3_4.csv")
# df1.Zone_id= df1.Zone_id.astype(str)
# df1.head(1)

In [ ]:
df1.shape

In [ ]:
beacon_cols = [col for col in df1.columns if str(col).startswith('0')]
rows_all_below_90 = df1[beacon_cols].lt(-99).all(axis=1)
df1 = df1[~rows_all_below_90]
df1.shape

In [ ]:
df1.Room_name.nunique()

### Check if dataset have enoguh beacon heard >=-90, SELECT ONLY the number of strong features >=5

In [ ]:
df2 = pd.merge(df1.drop(columns=["parent_zone_id", "beacon_count"]), \
                                ground_truth_df[['Zone_id','x', 'y']], on=["Zone_id"], how='left')
rssi_cols = [col for col in df2.columns if col.startswith('0')]

# Create a new column counting RSSIs >= -90
df2['num_strong_features'] = (df2[rssi_cols] >= -95).sum(axis=1)

df2.shape

In [ ]:
data_set_df = df2[df2['num_strong_features'] >= 5].copy()

data_set_df=data_set_df.drop(columns='num_strong_features').reset_index(drop=True)
data_set_df.shape

# *** MAKE SURE THAT ALL BEACONS ARE BEING HEARD

In [ ]:
# data_set_df.to_csv("data/Asset/data_asset_open_area_with_ground_truth_Nov24.csv", index=False)

In [ ]:
# data_set_df=pd.read_csv("data/Asset/data_asset_open_area_with_ground_truth_Nov24.csv")

In [ ]:
data_set_df.Room_name.nunique()

In [ ]:
data_set_df.shape

In [ ]:
anchor_point_df.shape

In [ ]:
len(data_set_df.columns.intersection(anchor_point_df.Mac))

# Location AI

In [ ]:
columns_to_drop= anchor_point_df[anchor_point_df.Remove=="remove"].Mac.tolist()
len(columns_to_drop)

In [ ]:
df2= pd.read_csv("data/Asset/Original data/2_data_asset_acess_Jun23_used_for_locAI_training.csv")
df2['Zone_id']=df2['Zone_id'].astype(str)
df2= df2[df1.columns]

df2= df2.drop(columns= columns_to_drop)

train_data, test_data = train_test_split(df2, test_size=0.2, random_state=42, \
                                         stratify=df2["Zone_id"])

X_train = train_data[[col for col in train_data.columns if col.startswith("0")]]
# y_train_floor = train_data['parent_zone_id'] 
y_train = train_data['Zone_id']

X_test = data_set_df[[col for col in train_data.columns if col.startswith("0")]] 
# y_test_floor = df1['parent_zone_id'] 
y_test = data_set_df['Zone_id'] 
df2.shape

In [ ]:
rf_room = RandomForestClassifier(n_estimators=200, random_state=42)
rf_room.fit(X_train, y_train)

predicted_rooms = rf_room.predict(X_test)

## Need to chnage the label of tag in right area as I placed the tag right at the border

In [ ]:
data_set_df.Zone_id= data_set_df.Zone_id.astype(str)
result_d = data_set_df[["Room_name", 'tagId', 'Zone_id']]\
    .merge(ground_truth_df[["Zone_id", "Room_Type"]], on = "Zone_id", how="left")
# result_d

In [ ]:
result_d["Prediction"] = predicted_rooms.astype(str)
result_d["Prediction"]= result_d["Prediction"]
result_d["Accuracy"] = np.where(result_d.Zone_id == result_d.Prediction, 100, 0)
result_d= result_d.rename(columns={"tagId": "Tag_id"})
result_d.head(1)

In [ ]:
result_d[result_d.Room_name=="Lab Area 1"]

In [ ]:
result_d.Accuracy.mean()

In [ ]:
result_d.groupby(["Room_name"])["Accuracy"].mean()

# Fused of Opt and MLE

In [ ]:
def euclidean_dist(point, points, height_diff=1.5):
    return np.sqrt(np.sum((points - point) ** 2, axis=1) + height_diff ** 2)

def generate_grid(center, resolution=5, radius=4):
    step = 1 / resolution
    x_vals = np.arange(center[0] - radius, center[0] + radius + step, step)
    y_vals = np.arange(center[1] - radius, center[1] + radius + step, step)
    xv, yv = np.meshgrid(x_vals, y_vals)
    return np.stack([xv.ravel(), yv.ravel()], axis=1)

def trilateration(coords, distances):
    def fun(x, coords, distances):
        weights = 1 / (distances + 1e-5)
        return weights * (np.linalg.norm(coords - x, axis=1) - distances)
    x0 = np.mean(coords, axis=0)
    result = least_squares(fun, x0, args=(coords, distances))
    return result.x if result.success else x0

def rssi_to_distance(rssi, A=-40, n=3.5, scale=0.8):
    return scale * np.exp((A - rssi) / (10 * n))

def filter_rssi(row, beacon_positions, rssi_threshold=-90):
    return {
        mac: row[mac]
        for mac in beacon_positions.keys()
        if mac in row.index and isinstance(row[mac], (int, float)) and row[mac] > rssi_threshold
    }

def localization_error(tag_position, beacons, distances):
    estimated_distances = np.linalg.norm(beacons - tag_position, axis=1)
    sigma = np.std(distances) + 1e-3
    weights = np.exp(- (distances ** 2) / (2 * sigma ** 2))
    return np.sum(weights * (estimated_distances - distances) ** 2)

def generate_expansion_area(initial_guess, std_dev=0.5, radius=0.8, num_points=500, range_box=2):
    num_gauss = int(num_points * 0.4)
    num_box = int(num_points * 0.3)
    num_circle = num_points - num_gauss - num_box
    box_points = np.random.uniform(-range_box, range_box, size=(num_box, 2)) + initial_guess
    gauss_points = np.random.normal(0, std_dev, size=(num_gauss, 2)) + initial_guess
    r = radius * np.sqrt(np.random.uniform(0, 1, num_circle))
    theta = np.random.uniform(0, 2 * np.pi, num_circle)
    circ_points = np.column_stack((initial_guess[0] + r * np.cos(theta), initial_guess[1] + r * np.sin(theta)))
    return np.vstack((gauss_points, box_points, circ_points))

def calculate_total_error_to_all_beacons(best_position, beacon_coords):
    distances = np.linalg.norm(beacon_coords - best_position, axis=1)
    return np.sum(distances)

def compute_likelihood_weighted(grid_coords, anchor_coords, rssi_values, T, n, sigma_noise=4, anchor_weights=None):
    if anchor_weights is None:
        anchor_weights = np.ones_like(rssi_values)
    diff = grid_coords[:, None, :] - anchor_coords[None, :, :]
    dists = np.sqrt(np.sum(diff ** 2, axis=2) + 1.5**2)
    pred_rssi = T - 10 * n * np.log10(dists + 1e-5)
    residuals = pred_rssi - rssi_values
    weighted_residuals = (residuals / sigma_noise)**2 * anchor_weights
    likelihood = np.exp(-0.5 * weighted_residuals)
    return np.prod(likelihood, axis=1)

def find_mle_params(P_j, d_ij, init_guess=[-40, 3]):
    def squared_error(params, dists, rssi):
        T_i, n_p = params
        valid_mask = rssi != -100
        pred_rssi = T_i - 10 * n_p * np.log10(dists + 1e-5)
        return np.sum((pred_rssi[valid_mask] - rssi[valid_mask]) ** 2)
    bounds = [(-100, -30), (2, 6)]
    result = minimize(squared_error, init_guess, args=(d_ij, P_j),
                      method='L-BFGS-B', bounds=bounds)
    return result.x if result.success else init_guess

# Fused Localization Function
# -------------------------------
def fused_localization_ref(data_df, anchor_point_df,
                       sigma_noise=4, coarse_res=2, fine_res=5, fine_radius=3,
                       rssi_threshold=-95, strong_rssi_threshold=-75,
                       top_k_anchors=5, roi_margin=8, top_coarse_points=200, topN_ratio=0.05,
                       map_x_bounds=(0, 65), map_y_bounds=(0, 28),
                       epsilon=1e-12,
                       min_beacons_opt=3, max_beacons_opt=20,
                       expansion_radius=1, expansion_points=100, top_expansion_points=15,
                       enable_refinement=True):
    
    results = []
    beacon_positions = anchor_point_df[["x","y","Mac"]].set_index("Mac")[["x","y"]].to_dict(orient="index")
    has_timestamp = 'timestamp' in data_df.columns

    for idx, row in tqdm(data_df.iterrows(), total=len(data_df)):
        # ---------------------------
        # Extract RSSI
        # ---------------------------
        rssis = row.drop(['Zone_id','Room_name','x','y','tagId','timestamp'], errors='ignore').values.astype(float)
        anchor_coords = anchor_point_df[['x','y']].values

        # ---------------------------
        # MLE
        # ---------------------------
        mask_mle = rssis > rssi_threshold
        signal_strengths = rssis[mask_mle]
        dp_coords = anchor_coords[mask_mle]

        if len(signal_strengths) < 1:
            signal_strengths = rssis
            dp_coords = anchor_coords

        strong_mask = signal_strengths > strong_rssi_threshold
        if np.sum(strong_mask) < 2:
            dp_coords_selected = dp_coords
            signal_strengths_selected = signal_strengths
        else:
            dp_coords_selected = dp_coords[strong_mask]
            signal_strengths_selected = signal_strengths[strong_mask]

        min_rssi, max_rssi = np.min(signal_strengths_selected), np.max(signal_strengths_selected)
        anchor_weights = (signal_strengths_selected - min_rssi + 1) / (max_rssi - min_rssi + 1e-5)
        sorted_idx = np.argsort(-signal_strengths_selected)
        top_k = min(top_k_anchors, len(sorted_idx))
        top_coords = dp_coords_selected[sorted_idx[:top_k]]

        x_min, y_min = np.min(top_coords, axis=0)
        x_max, y_max = np.max(top_coords, axis=0)
        x_min = max(x_min - roi_margin, map_x_bounds[0])
        x_max = min(x_max + roi_margin, map_x_bounds[1])
        y_min = max(y_min - roi_margin, map_y_bounds[0])
        y_max = min(y_max + roi_margin, map_y_bounds[1])

        coarse_grid = np.stack(np.meshgrid(np.arange(x_min, x_max, 1/coarse_res),
                                           np.arange(y_min, y_max, 1/coarse_res)), axis=-1).reshape(-1,2)

        strongest_coord = top_coords[0]
        dists_for_fit = euclidean_dist(strongest_coord, dp_coords_selected)
        T_global, n_global = find_mle_params(signal_strengths_selected, dists_for_fit)

        coarse_likelihoods = compute_likelihood_weighted(
            coarse_grid, dp_coords_selected, signal_strengths_selected,
            T_global, n_global, sigma_noise, anchor_weights
        )

        top_indices = np.argpartition(coarse_likelihoods, -top_coarse_points)[-top_coarse_points:]
        top_candidates = coarse_grid[top_indices]

        fine_candidates, fine_likelihoods = [], []
        for center in top_candidates:
            fine_grid = generate_grid(center, resolution=fine_res, radius=fine_radius)
            likelihoods_fine = compute_likelihood_weighted(
                fine_grid, dp_coords_selected, signal_strengths_selected,
                T_global, n_global, sigma_noise, anchor_weights
            )
            fine_candidates.append(fine_grid)
            fine_likelihoods.append(likelihoods_fine)

        fine_candidates = np.vstack(fine_candidates)
        fine_likelihoods = np.hstack(fine_likelihoods)
        fine_likelihoods += epsilon
        fine_likelihoods /= np.sum(fine_likelihoods)

        N = max(1, min(100, int(topN_ratio * len(fine_candidates))))
        top_idx = np.argpartition(fine_likelihoods, -N)[-N:]
        top_points = fine_candidates[top_idx]
        top_weights = fine_likelihoods[top_idx]
        top_weights /= np.sum(top_weights)
        pred_mle = np.average(top_points, axis=0, weights=top_weights)
        conf_mle = np.max(top_weights)

        # ---------------------------
        # Optimization
        # ---------------------------
        rssi_values_opt = dict(sorted(filter_rssi(row, beacon_positions, rssi_threshold).items(), key=lambda x: x[1], reverse=True))
        if len(rssi_values_opt) < min_beacons_opt:
            beacon_coords_opt = anchor_coords
            distances_opt = np.ones(anchor_coords.shape[0])
        else:
            beacon_coords_opt = np.array([list(beacon_positions[b].values()) for b in rssi_values_opt.keys()])
            distances_opt = np.array([rssi_to_distance(rssi) for rssi in rssi_values_opt.values()])

        close_beacons = beacon_coords_opt[distances_opt <= 1]
        if len(close_beacons) > 0:
            initial_guess_opt = close_beacons[np.argmin(distances_opt[distances_opt <= 1])]
        else:
            top_n = min(5, len(distances_opt))
            initial_guess_opt = trilateration(beacon_coords_opt[:top_n], distances_opt[:top_n])

        best_positions_opt, total_errors_opt, num_beacons_used = [], [], []
        for num_beacons in range(min_beacons_opt, min(len(distances_opt), max_beacons_opt)+1):
            top_n_coords = beacon_coords_opt[:num_beacons]
            top_n_dists = distances_opt[:num_beacons]

            expansion_area = generate_expansion_area(initial_guess_opt, radius=expansion_radius, num_points=expansion_points)
            quick_errors = np.array([localization_error(p, top_n_coords, top_n_dists) for p in expansion_area])
            filtered_expansion_area = expansion_area[np.argsort(quick_errors)[:top_expansion_points]]

            best_err, best_pos = float("inf"), None
            for point in filtered_expansion_area:
                res = minimize(localization_error, point, args=(top_n_coords, top_n_dists), method='L-BFGS-B', options={'maxiter':100})
                if res.success:
                    est_pos = res.x
                    total_err = np.sum(np.linalg.norm(top_n_coords - est_pos, axis=1))
                    if total_err < best_err:
                        best_err = total_err
                        best_pos = est_pos
            best_positions_opt.append(best_pos)
            total_errors_opt.append(best_err)
            num_beacons_used.append(num_beacons)

        # Final selection based on total error
        best_idx_opt = np.argmin([calculate_total_error_to_all_beacons(p, beacon_coords_opt) for p in best_positions_opt])
        pred_opt = best_positions_opt[best_idx_opt]
        conf_opt = 1 / (1 + total_errors_opt[best_idx_opt])

        # ---------------------------
        # Final Refinement (matches original code)
        # ---------------------------
        pre_refined_pos = pred_opt.copy()
        if enable_refinement:
            refinement_rssi_threshold = -75
            strong_rssi_indices = [i for i, rssi in enumerate(rssi_values_opt.values()) if rssi > refinement_rssi_threshold]

            if len(strong_rssi_indices) >= 3:
                filtered_coords = beacon_coords_opt[strong_rssi_indices]
                filtered_distances = distances_opt[strong_rssi_indices]

                result = minimize(
                    localization_error,
                    pred_opt,
                    args=(filtered_coords, filtered_distances),
                    method='L-BFGS-B',
                    options={'maxiter': 100, 'gtol': 1e-8, 'disp': False}
                )
                if result.success:
                    pred_opt = result.x

        refinement_shift = np.linalg.norm(pred_opt - pre_refined_pos)

        # ---------------------------
        # Fused Results
        # ---------------------------
        alpha_dynamic = conf_mle / (conf_mle + conf_opt)
        pred_fused_fixed = 0.5 * pred_mle + 0.5 * pred_opt
        pred_fused_dynamic = alpha_dynamic * pred_mle + (1-alpha_dynamic) * pred_opt

        results.append({
            'original_index': idx,
            'Zone_id': row.get('Zone_id', np.nan),
            'Room_name': row.get('Room_name', np.nan),
            'Tag_id': row.get('tagId', np.nan),
            'timestamp': row.get('timestamp', np.nan),
            'Predicted_MLE': pred_mle,
            'Predicted_Optimisation': pred_opt,
            'Predicted_NLOS': pred_fused_fixed,
            'Predicted_NLOS_Dynamic': pred_fused_dynamic,
            'MLE_Confidence': conf_mle,
            'Opt_Confidence': conf_opt,
            'Ground_Truth': np.array([row['x'], row['y']])
        })

    return pd.DataFrame(results)


In [ ]:
# filename= "data_asset_tag_stay_still_combine_3_4"
filename

In [ ]:
data_set_df[data_set_df.Room_name=="Lab Area 3"]["x"].nunique()

In [ ]:
data_set_df.groupby("Room_name").tagId.nunique()

In [ ]:
start_time = time.perf_counter() 

result= fused_localization_ref(data_set_df, anchor_point_df)

save_folder = "Result_Asset"
save_name = f"{filename.replace('.json', '_NLOS.csv')}" 
save_path = os.path.join(save_folder, save_name)

result.to_csv(save_path, index=False)


end_time = time.perf_counter() 

total_time = end_time - start_time
avg_time_per_row = total_time / len(data_set_df)
print(avg_time_per_row)

In [ ]:
result.shape, data_set_df.shape

In [ ]:
def plot_predicted_fused_dynamic(result_df, ground_truth_df, map_file_location,
                                 fused_cols=['Predicted_NLOS_Dynamic', 'Predicted_NLOS'],
                                 output_file="compare_plot_MLE_Optim_Fused.png"):
    """
    Plot predicted locations from MLE, Optimisation, and fused methods, showing inside-room accuracy.
    Computes overall accuracy and returns per-room statistics including inside-point counts and total points.
    Also provides room-type aggregated accuracy.
    """
    results = []

    merged_df = pd.merge(result_df, ground_truth_df, on=["Zone_id", "Room_name"], how="left")
    unique_rooms = merged_df['Room_name'].unique()

    n_rows = math.ceil(len(unique_rooms) / 2)
    fig, axes = plt.subplots(n_rows, 2, figsize=(14, 3 * n_rows))
    axes = axes.flatten()

    total_inside = {"MLE": 0, "Optimisation": 0}
    total_inside.update({col: 0 for col in fused_cols})
    total_points = 0  # only one total points count

    for i, room_name in enumerate(unique_rooms):
        room_data = merged_df[merged_df['Room_name'] == room_name]
        zone = room_data["Zone_id"].iloc[0]
        room_type = room_data["Room_Type"].iloc[0]
        room_box = room_data.iloc[0]

        # Room polygon
        x_coords = [room_box.get(f'x{i+1}', None) for i in range(8) if pd.notnull(room_box.get(f'x{i+1}', None))]
        y_coords = [room_box.get(f'y{i+1}', None) for i in range(8) if pd.notnull(room_box.get(f'y{i+1}', None))]
        coordinates = list(zip(x_coords, y_coords))
        polygon = Polygon(coordinates)
        if not polygon.is_valid:
            polygon = polygon.buffer(0)

        # Helper function to parse coordinates
        def parse_coords(col_name):
            x_list, y_list = [], []
            for coord in room_data[col_name]:
                try:
                    coord = ast.literal_eval(coord) if isinstance(coord, str) else coord
                    x_list.append(float(coord[0]))
                    y_list.append(float(coord[1]))
                except:
                    pass
            return x_list, y_list

        # Predictions
        x_mle, y_mle = parse_coords("Predicted_MLE")
        x_opt, y_opt = parse_coords("Predicted_Optimisation")
        fused_data = {col: parse_coords(col) for col in fused_cols if col in room_data.columns}

        # Count points inside polygon
        def count_inside(x_list, y_list):
            return sum(1 for x, y in zip(x_list, y_list) if Point(x, y).within(polygon))

        inside_mle = count_inside(x_mle, y_mle)
        inside_opt = count_inside(x_opt, y_opt)
        inside_fused = {k: count_inside(*v) for k, v in fused_data.items()}

        # Update totals
        total_inside["MLE"] += inside_mle
        total_inside["Optimisation"] += inside_opt
        for k, v in fused_data.items():
            total_inside[k] += inside_fused[k]
        total_points += len(x_mle)  # same for all methods

        # Save per-room results
        results.append({
            "Zone_id": zone,
            "Room_name": room_name,
            "Room_Type": room_type,
            "MLE_Accuracy": inside_mle / max(len(x_mle), 1) * 100,
            "Optimisation_Accuracy": inside_opt / max(len(x_opt), 1) * 100,
            **{f"{k}_Accuracy": inside_fused[k] / max(len(fused_data[k][0]), 1) * 100 for k in fused_data},
            "MLE_Inside_Points": inside_mle,
            "Optimisation_Inside_Points": inside_opt,
            **{f"{k}_Inside_Points": inside_fused[k] for k in fused_data},
            "Total_Points": len(x_mle)
        })

        # Plotting
        ax = axes[i]
        image = mpimg.imread(map_file_location)
        ax.imshow(image, extent=[0, 65, 0, 28], aspect='auto')
        ax.plot(x_coords + [x_coords[0]], y_coords + [y_coords[0]], 'r-', label='Room Boundary')
        ax.scatter(x_mle, y_mle, color='blue', s=8, label='MLE')
        ax.scatter(x_opt, y_opt, color='green', s=8, label='Optimisation')
        colors = ['orange', 'purple', 'red', 'cyan']
        for j, (fcol, (x_f, y_f)) in enumerate(fused_data.items()):
            ax.scatter(x_f, y_f, color=colors[j % len(colors)], s=8, label=f"{fcol}")

        ax.set_xlim([0, 65])
        ax.set_ylim([0, 28])
        
        # Build title using percentage accuracy instead of counts
        title_str = (
            f"{room_name} - "
            f"MLE: {inside_mle / max(len(x_mle), 1) * 100:.1f}%, "
            f"Opt: {inside_opt / max(len(x_opt), 1) * 100:.1f}%"
        )

        for fcol, (x_f, y_f) in fused_data.items():
            acc = inside_fused[fcol] / max(len(x_f), 1) * 100
            clean_name = fcol.replace("Predicted_", "")  # <--- removes the prefix
            title_str += f", {clean_name}: {acc:.1f}%"


            
        ax.set_title(title_str)
        ax.legend(loc='upper left', bbox_to_anchor=(0, 0), ncol=2)

    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    accuracy_df = pd.DataFrame(results)

    # --- Overall accuracy ---
    print("\n=== Overall Accuracy ===")
    for method in total_inside.keys():
        overall = total_inside[method] / max(total_points, 1) * 100
        print(f"{method}: {overall:.2f}%")

    # --- Room-type aggregated accuracy ---
    room_type_stats = accuracy_df.groupby('Room_Type').agg({
        'MLE_Inside_Points': 'sum',
        'Optimisation_Inside_Points': 'sum',
        **{f"{col}_Inside_Points": 'sum' for col in fused_cols},
        'Total_Points': 'sum'
    })

    for method in ['MLE', 'Optimisation'] + fused_cols:
        room_type_stats[f"{method}_Accuracy"] = room_type_stats[f"{method}_Inside_Points"] / room_type_stats['Total_Points'] * 100

    plt.tight_layout()
    plt.savefig(output_file, format="png")
    plt.show()

    return accuracy_df

from matplotlib.patches import Patch
from shapely.geometry import Polygon

In [ ]:
result.groupby("Room_name").Tag_id.nunique()

In [ ]:
accuracy_df = plot_predicted_fused_dynamic(
    result_df=result,
    ground_truth_df=ground_truth_df,
    map_file_location=map_file,
    fused_cols=['Predicted_NLOS'],
    output_file="compare_fused_results.png"
)

In [ ]:
accuracy_df[["Zone_id","Room_name", "Predicted_NLOS_Accuracy"]]

In [ ]:
def plot_all_zones_NLOS_only(result_df, ground_truth_df, map_file_location,
                             nlos_cols=['Predicted_NLOS_Dynamic', 'Predicted_NLOS'],
                             output_file="combined_all_zones_NLOS.png"):
    """
    Plot a single combined map of all zones:
    - Each zone shown in a unique color
    - Only NLOS prediction columns are plotted
    - Zone polygon borders included
    """

    # Merge results with ground truth to get coordinates and boundaries
    merged_df = pd.merge(result_df, ground_truth_df, on=["Zone_id", "Room_name"], how="left")

    fig, ax = plt.subplots(figsize=(14, 7))

    # Load background map
    image = mpimg.imread(map_file_location)
    ax.imshow(image, extent=[0, 65, 0, 28], aspect='auto')

    # Unique colors per zone
    unique_zones = merged_df['Zone_id'].unique()
    zone_colors = {z: plt.cm.tab20(i % 20) for i, z in enumerate(unique_zones)}

    # Marker shapes for each NLOS column (different markers, same zone color)
    markers = ["o", "s", "D", "X", "P", "^"]
    method_markers = {col: markers[i % len(markers)] for i, col in enumerate(nlos_cols)}

    # Helper to decode coordinate strings
    def parse_coords(col, df):
        x_list, y_list = [], []
        for coord in df[col]:
            try:
                coord = ast.literal_eval(coord) if isinstance(coord, str) else coord
                x_list.append(float(coord[0]))
                y_list.append(float(coord[1]))
            except:
                pass
        return x_list, y_list

    # ======== PLOT ALL ZONES ========
    for zone in unique_zones:
        zdata = merged_df[merged_df["Zone_id"] == zone]
        color = zone_colors[zone]

        # ---- Draw zone boundary (polygon) ----
        room_box = zdata.iloc[0]
        bx = [room_box.get(f'x{i+1}', None) for i in range(8) if pd.notnull(room_box.get(f'x{i+1}', None))]
        by = [room_box.get(f'y{i+1}', None) for i in range(8) if pd.notnull(room_box.get(f'y{i+1}', None))]

        if len(bx) > 2:
            ax.plot(bx + [bx[0]], by + [by[0]], color=color, linewidth=2)

        # ---- Plot NLOS predictions only ----
        for col in nlos_cols:
            if col in zdata.columns:
                x_vals, y_vals = parse_coords(col, zdata)
                ax.scatter(x_vals, y_vals, s=16, color=color,
                           marker=method_markers[col],
                           label=f"Zone {zone} – {col}")

    # Formatting
    ax.set_xlim([0, 65])
    ax.set_ylim([0, 28])
    ax.set_title("All Zones – NLOS Predictions Only (With Zone Borders)")

    # Combine legend (remove duplicates)
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
#     ax.legend(by_label.values(), by_label.keys(),
#               loc="upper left", bbox_to_anchor=(1.02, 1))

    plt.tight_layout()
    plt.savefig(output_file, dpi=300)
    plt.show()


In [ ]:
from matplotlib.patches import Patch
from shapely.geometry import Polygon

accuracy_df = plot_all_zones_NLOS_only(
    result_df=result_nlos,
    ground_truth_df=ground_truth_df,
    map_file_location=map_file,
    nlos_cols=['Predicted_NLOS'],
    output_file="compare_fused_results.png"
)

In [ ]:
tag_df= pd.read_csv("data/Asset/Open_area_with_ground_truth/Tag_ground_truth_data_Nov_24_mod.csv")
tag_df.Tag_id= tag_df.Tag_id.astype(str)
tag_df.head(1)

In [ ]:
import matplotlib.image as mpimg
import ast
import math
from matplotlib.lines import Line2D

def plot_all_zones_with_predictions(
    result_df,
    ground_truth_df,
    map_file_location,
    tag_df=None,
    classification_df=None,
    nlos_accuracy_df=None,
    nlos_cols=['Predicted_NLOS_Dynamic', 'Predicted_NLOS'],
    output_file="combined_all_zones.png",
    connect_tag_to_pred=True,
    connect_tag_to_classification=True,
    show_classification_accuracy=True
):
    """
    Plot zones with polygon boundaries, NLOS predictions, tags, and classification predictions.
    Only wrong classification predictions are connected for clarity.
    Uses dotted lines for classification connections with labels offset from line.
    """

    # Merge results with ground truth
    merged_df = pd.merge(
        result_df,
        ground_truth_df,
        on=["Zone_id", "Room_name"],
        how="left"
    )

    # Setup figure and background map
    fig, ax = plt.subplots(figsize=(10, 7))
    image = mpimg.imread(map_file_location)
    ax.imshow(image, extent=[0, 65, 0, 28], aspect='auto', alpha=0.8)

    # Color assignments for zones
    fixed_colors = [
        '#1f77b4', '#2ca02c','#ff7f0e',  '#d62728', '#9467bd',
        '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'
    ]
    unique_zones = sorted(merged_df['Zone_id'].unique())
    zone_colors = {z: fixed_colors[i % len(fixed_colors)] for i, z in enumerate(unique_zones)}

    markers = ["o", "s", "D", "X", "P", "^"]
    method_markers = {col: markers[i % len(markers)] for i, col in enumerate(nlos_cols)}

    def parse_coords(col, df):
        xs, ys = [], []
        for coord in df[col]:
            try:
                c = ast.literal_eval(coord) if isinstance(coord, str) else coord
                xs.append(float(c[0]))
                ys.append(float(c[1]))
            except Exception:
                continue
        return xs, ys

    # Draw zones & NLOS
    for zone in unique_zones:
        zdata = merged_df[merged_df["Zone_id"] == zone]
        color = zone_colors[zone]

        room_row = zdata.iloc[0]
        bx = [room_row.get(f'x{i}', None) for i in range(1, 9) if pd.notnull(room_row.get(f'x{i}', None))]
        by = [room_row.get(f'y{i}', None) for i in range(1, 9) if pd.notnull(room_row.get(f'y{i}', None))]
        if len(bx) > 2 and len(by) > 2:
            ax.plot(bx + [bx[0]], by + [by[0]], color=color, linewidth=1, zorder=2)

        for col in nlos_cols:
            if col in zdata.columns:
                xs, ys = parse_coords(col, zdata)
                ax.scatter(xs, ys, s=2, color=color, marker=method_markers[col], zorder=3)

    # Plot tags and NLOS lines
    if tag_df is not None:
        if 'Zone_id' not in tag_df.columns:
            tag_df = pd.merge(tag_df, ground_truth_df[['Room_name', 'Zone_id']],
                              on='Room_name', how='left')

        for _, row in tag_df.iterrows():
            tx, ty = row["x"], row["y"]
            tag_zone = row.get("Zone_id")
            tag_color = zone_colors.get(tag_zone, "red")

            ax.scatter(tx, ty, marker="*", s=180, color=tag_color,
                       edgecolors="black", linewidths=1.2, zorder=4)

#             # **Add tag label**
#             tag_label = row.get("Tag_id", "")
#             ax.text(tx + 0.2, ty + 0.2, str(tag_label), fontsize=8,
#                     color='black', weight='bold', zorder=5)
        
            if connect_tag_to_pred and tag_zone in unique_zones:
                zdata = merged_df[(merged_df['Zone_id'] == tag_zone) &
                                  (merged_df['Tag_id'] == row['Tag_id'])]
                for col in nlos_cols:
                    if col in zdata.columns:
                        xs, ys = parse_coords(col, zdata)
                        for px, py in zip(xs, ys):
                            ax.plot([tx, px], [ty, py], color=tag_color,
                                    alpha=0.4, linewidth=1, zorder=3)

    # Plot classification predictions (wrong only) with dotted lines
    if classification_df is not None and connect_tag_to_classification:
        grouped = classification_df.groupby(["Zone_id", "Tag_id"])
        for (zone_id, tag_id), group in grouped:

            tag_row = tag_df[(tag_df["Tag_id"] == tag_id) & (tag_df.get("Zone_id") == zone_id)]
            if tag_row.empty:
                continue
            tx, ty = float(tag_row.iloc[0]["x"]), float(tag_row.iloc[0]["y"])

            pred_counts = group["Prediction"].value_counts(normalize=True)

            for pred_zone, fraction in pred_counts.items():
                # Skip correct prediction for dotted line
                if pred_zone == zone_id:
                    continue

                # Centroid of predicted zone
                zone_row = ground_truth_df[ground_truth_df["Zone_id"] == pred_zone]
                if zone_row.empty:
                    continue
                x_cols = [f"x{i}" for i in range(1, 9)]
                y_cols = [f"y{i}" for i in range(1, 9)]
                xs = [zone_row.iloc[0][c] for c in x_cols if pd.notnull(zone_row.iloc[0][c])]
                ys = [zone_row.iloc[0][c] for c in y_cols if pd.notnull(zone_row.iloc[0][c])]
                if not xs or not ys:
                    continue
                cx, cy = sum(xs)/len(xs), sum(ys)/len(ys)

                # Dotted line to wrong prediction
                ax.plot([tx, cx], [ty, cy], color="black", alpha=0.5,
                        linewidth=1.2, linestyle='dotted', zorder=4)

                # Place percentage + predicted zone label slightly above line
                offset = 0.2
                mid_x, mid_y = (tx + cx)/2, (ty + cy)/2
                dx, dy = cx - tx, cy - ty
                angle = math.degrees(math.atan2(dy, dx))
                label_x = mid_x - dy/abs(dy+1e-6)*offset if dy != 0 else mid_x
                label_y = mid_y + dx/abs(dx+1e-6)*offset if dx != 0 else mid_y + offset
                ax.text(label_x, label_y, f"{fraction*100:.1f}%",
                        fontsize=7, color="b",  zorder=6,
                        ha="center", va="center", rotation=angle,weight="bold",
                        rotation_mode="anchor",  alpha=0.7) # backgroundcolor="white", weight="bold",

                # Triangle at predicted centroid
                ax.scatter(cx, cy, marker="^", s=200,
                           color=zone_colors.get(pred_zone, "yellow"),
                           edgecolors="black", linewidths=1.2, zorder=5)

    # Optional: per-zone accuracy labels
    classification_accuracy = {}
    if classification_df is not None:
        classification_df['Correct'] = classification_df['Prediction'] == classification_df['Zone_id']
        classification_accuracy = classification_df.groupby('Zone_id')['Correct'].mean() * 100
        classification_accuracy = classification_accuracy.to_dict()

    nlos_accuracy_dict = {}
    if nlos_accuracy_df is not None:
        nlos_accuracy_dict = dict(zip(nlos_accuracy_df['Zone_id'], nlos_accuracy_df['Predicted_NLOS_Accuracy']))

    for zone in unique_zones:
        zone_row = ground_truth_df[ground_truth_df['Zone_id'] == zone]
        if zone_row.empty:
            continue
        x_cols = [f'x{i}' for i in range(1, 9)]
        y_cols = [f'y{i}' for i in range(1, 9)]
        xs = [zone_row.iloc[0][c] for c in x_cols if pd.notnull(zone_row.iloc[0][c])]
        ys = [zone_row.iloc[0][c] for c in y_cols if pd.notnull(zone_row.iloc[0][c])]
        if not xs or not ys:
            continue
        cx, cy = sum(xs)/len(xs), sum(ys)/len(ys)

        label_text = ""
        if show_classification_accuracy:
            cls_acc = classification_accuracy.get(zone, np.nan)
            label_text += f"LocAI: {cls_acc:.1f}%\n"
        nlos_acc = nlos_accuracy_dict.get(zone, np.nan)
        label_text += f"NLOS: {nlos_acc:.1f}%"

        ax.text(cx, cy - 0.5, label_text,
                fontsize=10, color='black', ha='center', va='bottom', weight='bold',
                zorder=6, bbox=dict(facecolor='white', alpha=0.6, edgecolor='none', pad=1))

    # Legend & formatting
    ax.set_xlim([30, 65])
    ax.set_ylim([0, 28])
    ax.set_title("NLOS & LocationAI Predictions", fontsize=14)

    legend_elements = [
        Line2D([0], [0], marker='*', color='w', label='Tag Location',
               markerfacecolor='black', markersize=15, markeredgecolor='black'),
        Line2D([0], [0], marker='^', color='w', label='LocAI Prediction',
               markerfacecolor='gray', markersize=12, markeredgecolor='black'),
        Line2D([0], [0], marker='.', color='w', label='NLOS Prediction',
               markerfacecolor='gray', markersize=12, markeredgecolor='black')
    ]
    ax.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(0.22, 1))

    plt.tight_layout()
    plt.savefig(output_file, dpi=300)
    plt.show()


In [ ]:
# accuracy_df

In [ ]:
zone= ['Lab Area 2','Lab Area 3',  'Lab Area 1', 'Lounge Area']
# zone= ['Lab Area 1',, '30537']
plot_all_zones_with_predictions(
    result[result.Room_name.isin(zone)],
    ground_truth_df,
    map_file,
    tag_df=tag_df[tag_df.Room_name.isin(zone)],
    classification_df=result_d[result.Room_name.isin(zone)],
    nlos_accuracy_df=accuracy_df,
    nlos_cols=['Predicted_NLOS'],
    output_file="combined_all_zones.png",
    connect_tag_to_pred=True,
    connect_tag_to_classification=True
)

In [ ]:
result_df= result[["Zone_id", "Room_name", "Tag_id", "Predicted_NLOS"]]

result_df['Tag_id'] = result_df['Tag_id'].astype(str)
tag_df['Tag_id'] = tag_df['Tag_id'].astype(str)

# Merge to get tag x, y
merged_los = pd.merge(
    result_df,
    tag_df[['Tag_id', 'x', 'y', 'Room_name']],
    on=['Tag_id', 'Room_name'],
    how='left',
    suffixes=('', '_tag')
)

In [ ]:
def euclidean_distance(pt1, pt2):
    try:
        return np.sqrt((pt1[0]-pt2[0])**2 + (pt1[1]-pt2[1])**2)
    except:
        return np.nan

# Compute distance between Predicted_MLE or Predicted_NLOS and tag location
merged_los['LOS_Error'] = merged_los.apply(
    lambda row: euclidean_distance(row['Predicted_NLOS'], [row['x'], row['y']]),
    axis=1
)
# merged_los

In [ ]:
# Ensure Tag_id is string in both dataframes
result_d['Tag_id'] = result_d['Tag_id'].astype(str)
tag_df['Tag_id'] = tag_df['Tag_id'].astype(str)

# Merge to get tag x, y coordinates
merged_locai = pd.merge(
    result_d,
    tag_df[['Tag_id', 'x', 'y', 'Room_name']],
    on=['Tag_id', 'Room_name'],
    how='left'
)
# merged_locai

In [ ]:
def get_zone_centroid(zone_id, gt_df):
    zone_row = gt_df[gt_df['Zone_id'] == zone_id]
    if zone_row.empty:
        return np.nan, np.nan
    x_cols = [f'x{i}' for i in range(1, 9)]
    y_cols = [f'y{i}' for i in range(1, 9)]
    xs = [zone_row.iloc[0][c] for c in x_cols if pd.notnull(zone_row.iloc[0][c])]
    ys = [zone_row.iloc[0][c] for c in y_cols if pd.notnull(zone_row.iloc[0][c])]
    if not xs or not ys:
        return np.nan, np.nan
    return np.mean(xs), np.mean(ys)

def euclidean_distance(pt1, pt2):
    try:
        return np.sqrt((pt1[0]-pt2[0])**2 + (pt1[1]-pt2[1])**2)
    except:
        return np.nan

# Compute centroid of predicted zone and distance to tag
def compute_location_ai_error(row, gt_df):
    pred_cx, pred_cy = get_zone_centroid(row['Prediction'], gt_df)
    if np.isnan(pred_cx) or np.isnan(pred_cy):
        return np.nan
    tag_x, tag_y = row['x'], row['y']
    return euclidean_distance([pred_cx, pred_cy], [tag_x, tag_y])

merged_locai['LocationAI_Error'] = merged_locai.apply(lambda row: compute_location_ai_error(row, ground_truth_df), axis=1)
merged_locai.head(1)

In [ ]:
los_errors = merged_los['LOS_Error'].dropna().values
locai_errors = merged_locai['LocationAI_Error'].dropna().values

# Sort and compute CDF
los_sorted = np.sort(los_errors)
los_cdf = np.arange(1, len(los_sorted)+1) / len(los_sorted)

locai_sorted = np.sort(locai_errors)
locai_cdf = np.arange(1, len(locai_sorted)+1) / len(locai_sorted)


In [ ]:
def get_value_at_cdf(x, cdf, percentile):
    # percentile in [0,1]
    return np.interp(percentile, cdf, x)


In [ ]:
# LOS
los_50_val = get_value_at_cdf(los_sorted, los_cdf, 0.5)
los_90_val = get_value_at_cdf(los_sorted, los_cdf, 0.9)

# LocationAI
locai_50_val = get_value_at_cdf(locai_sorted, locai_cdf, 0.5)
locai_90_val = get_value_at_cdf(locai_sorted, locai_cdf, 0.9)


In [ ]:
offset = 0.3  # meters to shift text left/right

# Convert CDF to percent
los_cdf_percent = los_cdf * 100
locai_cdf_percent = locai_cdf * 100

# Plot CDFs
plt.plot(los_sorted, los_cdf_percent, label='NLOS Error', marker='o', linestyle='-', alpha=0.1)
plt.plot(locai_sorted, locai_cdf_percent, label='LocationAI Error', marker='s', linestyle='--', alpha=0.1)

# Horizontal lines at 50% and 90%
for y in [50, 90]:
    plt.axhline(y=y, color='gray', linestyle=':', alpha=0.7)

# Annotate x-values for 50% CDF
plt.text(los_50_val - offset, 50+2, f'{los_50_val:.1f}m', color='blue', ha='right', va='bottom')
plt.text(locai_50_val + offset, 50+2, f'{locai_50_val:.1f}m', color='red', ha='right', va='bottom')

# Annotate x-values for 90% CDF
plt.text(los_90_val - offset, 90+2, f'{los_90_val:.1f}m', color='blue', ha='right', va='bottom')
plt.text(locai_90_val + offset, 90+2, f'{locai_90_val:.1f}m', color='red', ha='right', va='bottom')

plt.xlabel('Distance Error, meters')
plt.ylabel('%')
plt.title('CDF of Distance Errors of 4 Open Areas with 50% and 90% Confidence')
plt.grid(False)
plt.legend()
plt.ylim(0, 105)  # add some space above 100%
plt.tight_layout()
plt.show()
